In [1]:
import boto3
import re
import logging
from botocore.exceptions import BotoCoreError, ClientError

# Validates the format of an email address using a regular expression
def validate_email(email):
    pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return re.match(pattern, email)

# Creates an SNS topic and returns its ARN
def create_sns_topic(sns, topic_name):
    try:
        response = sns.create_topic(Name=topic_name)
        return response['TopicArn']
    except (BotoCoreError, ClientError) as error:
        logging.error(f"Error creating SNS topic: {error}")
        raise

# Subscribes an email address to a given SNS topic ARN
def subscribe_email(sns, topic_arn, email):
    try:
        sns.subscribe(
            TopicArn=topic_arn,
            Protocol='email',
            Endpoint=email
        )
    except (BotoCoreError, ClientError) as error:
        logging.error(f"Error subscribing email to SNS topic: {error}")
        raise

# Checks if a CloudWatch alarm with the given name already exists
def alarm_exists(cloudwatch, alarm_name):
    try:
        alarms = cloudwatch.describe_alarms(AlarmNames=[alarm_name])
        return len(alarms['MetricAlarms']) > 0
    except (BotoCoreError, ClientError) as error:
        logging.error(f"Error checking existing alarms: {error}")
        raise

# Creates a CloudWatch alarm to monitor estimated charges and trigger notifications
def create_cloudwatch_alarm(cloudwatch, topic_arn, threshold):
    try:
        cloudwatch.put_metric_alarm(
            AlarmName='BillingAlarm',
            MetricName='EstimatedCharges',
            Namespace='AWS/Billing',
            Statistic='Maximum',
            Period=21600,  # 6 hours
            EvaluationPeriods=1,
            Threshold=threshold,
            ComparisonOperator='GreaterThanThreshold',
            AlarmActions=[topic_arn],
            Dimensions=[
                {'Name': 'Currency', 'Value': 'CAD'}
            ]
        )
    except (BotoCoreError, ClientError) as error:
        logging.error(f"Error creating CloudWatch billing alarm: {error}")
        raise

# Main function to orchestrate the billing alarm setup
def main():
    logging.basicConfig(level=logging.INFO)
    print("AWS Billing Alarm Setup Script")

    # Gather and validate user inputs
    email = input("Enter the email address to receive alarm notifications: ").strip()
    while not validate_email(email):
        print("Invalid email format. Please try again.")
        email = input("Enter the email address to receive alarm notifications: ").strip()

    try:
        threshold = float(input("Enter the billing threshold (in CAD) to trigger the alarm: ").strip())
        if threshold <= 0:
            raise ValueError("Threshold must be a positive number.")
    except ValueError as e:
        print(f"Invalid threshold: {e}")
        return

    # Initialize Boto3 clients with a fixed region
    region = 'us-east-1'
    sns = boto3.client('sns', region_name=region)
    cloudwatch = boto3.client('cloudwatch', region_name=region)

    # Create SNS Topic
    print("Creating SNS topic...")
    topic_name = "BillingAlarmTopic"
    try:
        topic_arn = create_sns_topic(sns, topic_name)
        print(f"SNS Topic created with ARN: {topic_arn}")
    except Exception:
        print("Failed to create SNS topic.")
        return

    # Subscribe email to the SNS Topic
    print(f"Subscribing {email} to the SNS topic...")
    try:
        subscribe_email(sns, topic_arn, email)
        print(f"Subscription confirmation email sent to {email}. Please confirm the subscription.")
    except Exception:
        print("Failed to subscribe email to SNS topic.")
        return

    # Check if the alarm already exists
    alarm_name = 'BillingAlarm'
    print("Checking for existing billing alarms...")
    if alarm_exists(cloudwatch, alarm_name):
        print(f"An alarm with the name '{alarm_name}' already exists. Please confirm if you want to overwrite it.")
        confirm = input("Do you want to overwrite the existing alarm? (yes/no): ").strip().lower()
        if confirm != 'yes':
            print("Aborting alarm creation.")
            return

    # Create the CloudWatch billing alarm
    print("Creating CloudWatch billing alarm...")
    try:
        create_cloudwatch_alarm(cloudwatch, topic_arn, threshold)
        print(f"Billing alarm created successfully with a threshold of ${threshold:.2f} CAD.")
    except Exception:
        print("Failed to create CloudWatch billing alarm.")
        return

if __name__ == "__main__":
    main()


AWS Billing Alarm Setup Script


Enter the email address to receive alarm notifications:  cahills@queensu.ca
Enter the billing threshold (in CAD) to trigger the alarm:  35


INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


Creating SNS topic...
SNS Topic created with ARN: arn:aws:sns:us-east-1:525897591902:BillingAlarmTopic
Subscribing cahills@queensu.ca to the SNS topic...
Subscription confirmation email sent to cahills@queensu.ca. Please confirm the subscription.
Checking for existing billing alarms...
An alarm with the name 'BillingAlarm' already exists. Please confirm if you want to overwrite it.


Do you want to overwrite the existing alarm? (yes/no):  yes


Creating CloudWatch billing alarm...
Billing alarm created successfully with a threshold of $35.00 CAD.
